In [2]:
import numpy as np
import keras
import keras.backend as k
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
from keras.optimizers import Adam
from keras.preprocessing import image
import cv2
import datetime

In [3]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [6]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('C:/Users/HEWLLET/PycharmProjects/Homework/train',
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'C:/Users/HEWLLET/PycharmProjects/Homework/train',
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')

model_saved=model.fit_generator(
        training_set,
        epochs=10,
        validation_data=test_set,

        )

Found 1376 images belonging to 2 classes.
Found 1376 images belonging to 2 classes.


C:\Users\HEWLLET\AppData\Local\Temp\ipykernel_6216\2580032022.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_saved=model.fit_generator(


Epoch 1/10
86/86 [==============================] - 42s 462ms/step - loss: 0.3323 - accuracy: 0.8336 - val_loss: 0.1740 - val_accuracy: 0.9513
Epoch 2/10
86/86 [==============================] - 40s 459ms/step - loss: 0.1294 - accuracy: 0.9564 - val_loss: 0.0695 - val_accuracy: 0.9818
Epoch 3/10
86/86 [==============================] - 39s 455ms/step - loss: 0.0813 - accuracy: 0.9717 - val_loss: 0.0504 - val_accuracy: 0.9855
Epoch 4/10
86/86 [==============================] - 39s 454ms/step - loss: 0.0618 - accuracy: 0.9818 - val_loss: 0.0355 - val_accuracy: 0.9876
Epoch 5/10
86/86 [==============================] - 39s 454ms/step - loss: 0.0523 - accuracy: 0.9833 - val_loss: 0.0350 - val_accuracy: 0.9913
Epoch 6/10
86/86 [==============================] - 39s 454ms/step - loss: 0.0627 - accuracy: 0.9782 - val_loss: 0.0351 - val_accuracy: 0.9862
Epoch 7/10
86/86 [==============================] - 39s 452ms/step - loss: 0.0553 - accuracy: 0.9811 - val_loss: 0.0258 - val_accuracy: 0.9913

In [13]:
model.save('mymodel.h5',model_saved)

#To test for individual images

mymodel=load_model('mymodel.h5')
#test_image=image.load_img('C:/Users/Karan/Desktop/ML Datasets/Face Mask Detection/Dataset/test/without_mask/30.jpg',target_size=(150,150,3))
test_image=load_img(r'Screenshot_6.png',
                          target_size=(150,150,3))
test_image
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
mymodel.predict(test_image)[0][0]

1/1 [==============================] - 0s 191ms/step


0.0

In [8]:
from tensorflow.keras.utils import load_img

In [12]:
import keras.utils as image

In [ ]:
mymodel=load_model('mymodel.h5')

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()